# Laboratory exercise 6

The notebook contains exercises connected to auditory exercise 7. For any questions feel free to contact assistant: eda.jovicic@fer.hr

The main task of this notebook is to make predictions using supported learning with the Scikit Learn library. The goal is to predict the grade in Math considering other features of the student.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

1. Load the cleaned dataset from the first exercise. If you haven't saved the dataset, rerun the exercise and save the final dataset.

In [2]:
#solution
#loading data
df = pd.read_csv("dataset2.csv")

df

,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore,Gender_female,Gender_male,LunchType_free/reduced,LunchType_standard
0,female,2,1,standard,1,1,1,1,3.0,1,1,71,71,74,1.0,0.0,0.0,1.0
1,female,2,5,standard,1,1,2,1,0.0,2,0,69,90,88,1.0,0.0,0.0,1.0
2,female,1,3,standard,1,2,2,1,4.0,1,1,87,93,91,1.0,0.0,0.0,1.0
3,male,0,0,free/reduced,1,1,0,0,1.0,2,0,45,56,42,0.0,1.0,1.0,0.0
4,male,2,5,standard,1,1,2,1,0.0,1,0,76,78,75,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29112,female,3,2,standard,1,2,2,0,2.0,1,0,59,61,65,1.0,0.0,0.0,1.0
29113,male,4,2,standard,1,2,1,0,1.0,0,0,58,53,51,0.0,1.0,0.0,1.0
29114,female,5,2,free/reduced,0,1,2,0,1.0,0,0,61,70,67,1.0,0.0,1.0,0.0
29115,female,3,0,standard,0,1,1,0,3.0,1,0,82,90,93,1.0,0.0,0.0,1.0


2. Transform the MathScore feature into grades (1-5) using the following scoring system:

|   MathScore |  Grade  |
| ----------- | ------- |
|  88 - 100   |    5    |
|  75 - 87    |    4    |
|  63 - 74    |    3    |
|  50 - 62    |    2    |
|   0 - 49    |    1    |

In [3]:
#solution

score_bins = [-1, 49, 62, 74, 87, 100]
grade_labels = [1, 2, 3, 4, 5]

# Creating a new column 'Grade' based on the scoring system
df['Grade'] = pd.cut(df['MathScore'], bins=score_bins, labels=grade_labels, include_lowest=True)

# Displaying the changes
print(df[['MathScore', 'Grade']])

       MathScore Grade
0             71     3
1             69     3
2             87     4
3             45     1
4             76     4
...          ...   ...
29112         59     2
29113         58     2
29114         61     2
29115         82     4
29116         64     3

[29117 rows x 2 columns]


In [4]:
#replacing entries in Gender and LunchType so I would be able to do the calculations later
df['Gender'].replace({'male': 0, 'female':1}, inplace=True)

df['LunchType'].replace({'standard': 0, 'free/reduced':1}, inplace=True)

df

,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore,Gender_female,Gender_male,LunchType_free/reduced,LunchType_standard,Grade
0,1,2,1,0,1,1,1,1,3.0,1,1,71,71,74,1.0,0.0,0.0,1.0,3
1,1,2,5,0,1,1,2,1,0.0,2,0,69,90,88,1.0,0.0,0.0,1.0,3
2,1,1,3,0,1,2,2,1,4.0,1,1,87,93,91,1.0,0.0,0.0,1.0,4
3,0,0,0,1,1,1,0,0,1.0,2,0,45,56,42,0.0,1.0,1.0,0.0,1
4,0,2,5,0,1,1,2,1,0.0,1,0,76,78,75,0.0,1.0,0.0,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29112,1,3,2,0,1,2,2,0,2.0,1,0,59,61,65,1.0,0.0,0.0,1.0,2
29113,0,4,2,0,1,2,1,0,1.0,0,0,58,53,51,0.0,1.0,0.0,1.0,2
29114,1,5,2,1,0,1,2,0,1.0,0,0,61,70,67,1.0,0.0,1.0,0.0,2
29115,1,3,0,0,0,1,1,0,3.0,1,0,82,90,93,1.0,0.0,0.0,1.0,4


3. Divide the dataset into features (X) and predictions (y). For features we will use all the columns except MathScore, ReadingScore and WritingScore. For predictions we will use the MathScore column. Split the dataset into training and testing sets. The split should be done in a  70-30% ratio.

In [5]:
#df (all features except MathScore, ReadingScore, WritingScore) and y (MathScore)
feature_cols = ['Gender', 'EthnicGroup', 'ParentEduc', 'LunchType', 'TestPrep', 'ParentMaritalStatus', 'PracticeSport', 'IsFirstChild', 'NrSiblings', 'TransportMeans','WklyStudyHours',]

X = df[feature_cols]
y = df.Grade


In [6]:
#splitting the dataset into train and test (70-30% ratio)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

4. Create a Random Forest model, with max_depth=5 and n_estimators=20. Train the model using training set, and then test it on testing set. Display the confusion matrix. Show precision, recall and F1 score for all grades. 

In [7]:
#Random Forest model
clf = RandomForestClassifier(max_depth=5, random_state=0, n_estimators=20)
clf.fit(X_train, y_train)
# train_test_split ide sa clf.predict, predikcija se radi iz X_test skupa
y_pred = clf.predict(X_test)

# Display the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Display precision, recall, and F1 score for all grades
class_report = classification_report(y_test, y_pred)
print("\nClassification Report:")
print(class_report)


Confusion Matrix:
[[  68  676  412   18    0]
 [  32  868 1199  101    0]
 [  11  570 1700  266    0]
 [   1  262 1363  401    0]
 [   0   22  466  300    0]]

Classification Report:
              precision    recall  f1-score   support

           1       0.61      0.06      0.11      1174
           2       0.36      0.39      0.38      2200
           3       0.33      0.67      0.44      2547
           4       0.37      0.20      0.26      2027
           5       0.00      0.00      0.00       788

    accuracy                           0.35      8736
   macro avg       0.33      0.26      0.24      8736
weighted avg       0.35      0.35      0.30      8736



C:\Users\Mirela\Documents\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Mirela\Documents\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Mirela\Documents\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5. The accuracy of our model is not the best. The reason could be having too many possible classes (grades). Let's transform the data again, but this time, instead of predicting grades, we want to predict whether the student will pass (grades 2, 3, 4 and 5) or fail (grade 1) math. After transforming the MathScore accordingly (0 - failed, 1 - passed), repeat task 4 and compare the results.

In [8]:
#transform grades into failed (0) or passed (1)
df['Grade'] = pd.cut(df['MathScore'], bins=[0, 50, 101], labels=[0, 1], include_lowest=True)
df

,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore,Gender_female,Gender_male,LunchType_free/reduced,LunchType_standard,Grade
0,1,2,1,0,1,1,1,1,3.0,1,1,71,71,74,1.0,0.0,0.0,1.0,1
1,1,2,5,0,1,1,2,1,0.0,2,0,69,90,88,1.0,0.0,0.0,1.0,1
2,1,1,3,0,1,2,2,1,4.0,1,1,87,93,91,1.0,0.0,0.0,1.0,1
3,0,0,0,1,1,1,0,0,1.0,2,0,45,56,42,0.0,1.0,1.0,0.0,0
4,0,2,5,0,1,1,2,1,0.0,1,0,76,78,75,0.0,1.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29112,1,3,2,0,1,2,2,0,2.0,1,0,59,61,65,1.0,0.0,0.0,1.0,1
29113,0,4,2,0,1,2,1,0,1.0,0,0,58,53,51,0.0,1.0,0.0,1.0,1
29114,1,5,2,1,0,1,2,0,1.0,0,0,61,70,67,1.0,0.0,1.0,0.0,1
29115,1,3,0,0,0,1,1,0,3.0,1,0,82,90,93,1.0,0.0,0.0,1.0,1


In [9]:
#Random Forest model
X = df[feature_cols]
y = df.Grade
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_model = RandomForestClassifier(max_depth=5, n_estimators=20, random_state=42)
rf_model.fit(X,y)

# Predict on the testing set
y_pred = rf_model.predict(X_test)

# Display confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

# Display precision, recall, and F1 score for all grades
class_report = classification_report(y_test, y_pred)
print("\nClassification Report:")
print(class_report)

[[   0 1298]
 [   0 7438]]

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1298
           1       0.85      1.00      0.92      7438

    accuracy                           0.85      8736
   macro avg       0.43      0.50      0.46      8736
weighted avg       0.72      0.85      0.78      8736



C:\Users\Mirela\Documents\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Mirela\Documents\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Mirela\Documents\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


6. Compare the results. Did our model work better in the first case or the second? Explain why and suggest a way to improve it.

In the first case, the model is always incorrect when predicting grade 5, and in the second case, the model is always incorrect when predicting failing at Math exam. Overall, the second model is better at predicting passing, therefore the general accuracy is higher than in the first model. It is possible we have imbalanced dataset (classes of different size). We should consider oversampling the minority class, undersampling the majority class, or using different class weights during training. Also, we could adjust the complexity of the model. A more complex model may capture intricate patterns in the data, but it could also lead to overfitting. Generally, more decision trees give more accurate model, but we should see which number would be optimal for our model (in this example, the number of trees was set to 20).